In [ ]:
from lerobot.configs.eval import EvalPipelineConfig
from lerobot.configs.default import EvalConfig
from lerobot.envs.configs import LiberoEnv
from lerobot.policies.pi05.configuration_pi05 import PI05Config
from lerobot.policies.factory import make_policy, make_pre_post_processors


In [19]:
cfg = EvalPipelineConfig(
    env=LiberoEnv(),
    eval=EvalConfig(
        batch_size=1,
        n_episodes=1
    ),
    policy=PI05Config(n_action_steps=10),
    output_dir='./libero_eval_logs',
    job_name='test'
)

In [17]:
cfg.eval

EvalConfig(n_episodes=1, batch_size=1, use_async_envs=False)

In [1]:
# cp train_config.json /mnt/lustre-grete/usr/u12045/vla/hf_cache/hub/models--lerobot--pi0/snapshots/8f50aacbe079a026391616cf22453de528f2a873

In [1]:
import os
os.environ["HF_HOME"] = "/pfss/mlde/workspaces/mlde_wsp_PI_Hauschild/VLA/hf_cache"
os.environ["PATH"] = "/pfss/mlde/workspaces/mlde_wsp_PI_Hauschild/miniconda3/envs/pitorch/bin:" + os.environ.get("PATH", "")
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTHONPATH"] = "/pfss/mlde/workspaces/mlde_wsp_PI_Hauschild/VLA/duc/VLA-Humanoid:" + os.environ.get("PYTHONPATH", "")
import sys
sys.path.insert(0, "/pfss/mlde/workspaces/mlde_wsp_PI_Hauschild/VLA/duc/VLA-Humanoid")

# import os
# os.environ["HF_HOME"] = "/projects/extern/kisski/kisski-umg-fairpact-2/dir.project/VLA/hf_cache"
# os.environ["PATH"] = "/projects/extern/kisski/kisski-umg-fairpact-2/dir.project/miniconda3/envs/pitorch/bin:" + os.environ.get("PATH", "")
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["PYTHONPATH"] = "/projects/extern/kisski/kisski-umg-fairpact-2/dir.project/VLA/duc3/VLA-Humanoid:" + os.environ.get("PYTHONPATH", "")
# import sys
# sys.path.insert(0, "/projects/extern/kisski/kisski-umg-fairpact-2/dir.project/VLA/duc3/VLA-Humanoid")

import logging
import time
from contextlib import nullcontext
from pprint import pformat
from typing import Any

import torch
from termcolor import colored
from torch.amp import GradScaler
from torch.optim import Optimizer

from lerobot.common.datasets.factory import make_dataset
from lerobot.common.datasets.sampler import EpisodeAwareSampler
from lerobot.common.datasets.utils import cycle
from lerobot.common.envs.factory import make_env
from lerobot.common.optim.factory import make_optimizer_and_scheduler
from lerobot.common.policies.factory import make_policy
from lerobot.common.policies.pretrained import PreTrainedPolicy
from lerobot.common.policies.utils import get_device_from_parameters
from lerobot.common.utils.logging_utils import AverageMeter, MetricsTracker
from lerobot.common.utils.random_utils import set_seed
from lerobot.common.utils.train_utils import (
    get_step_checkpoint_dir,
    get_step_identifier,
    load_training_state,
    save_checkpoint,
    update_last_checkpoint,
)
from lerobot.common.utils.utils import (
    format_big_number,
    get_safe_torch_device,
    has_method,
    init_logging,
)
from lerobot.common.utils.wandb_utils import WandBLogger
from lerobot.configs import parser
from lerobot.configs.train import TrainPipelineConfig
from lerobot.scripts.eval import eval_policy
from lerobot.common.constants import ACTION, OBS_STATE

import json
from dotenv import load_dotenv
load_dotenv()


%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'lerobot.common'

In [2]:
init_logging()

In [7]:
from lerobot.configs.policies import PreTrainedConfig
from lerobot.configs.default import DatasetConfig, EvalConfig, WandBConfig
from lerobot.common.datasets.transforms import ImageTransformsConfig

with open("../configs/libero_config/default.json") as f:
    data = json.load(f)
# /mnt/lustre-grete/usr/u12045/vla/hf_cache/lerobot/binhng/libero_spatial_mask_debug_noops_lerobot
cfg = TrainPipelineConfig(
    policy=PreTrainedConfig.from_pretrained("/pfss/mlde/workspaces/mlde_wsp_PI_Hauschild/VLA/temp/baseline"),
    dataset=DatasetConfig(
        repo_id="None",
        root="/pfss/mlde/workspaces/mlde_wsp_PI_Hauschild/VLA/LIBERO/merged_libero_scale_10_mask_depth_noops_lerobot_original",
        image_transforms=ImageTransformsConfig.from_dict(data["dataset"]['image_transforms']),
    ),
    wandb=WandBConfig(
        project="lerobot",
        entity="lerobot",
    ),
)
cfg.validate()
# logging.info(pformat(cfg.to_dict()))

In [8]:
cfg.policy.chunk_size = 10
cfg.policy.n_action_steps = 10
cfg.batch_size = 1
cfg.wandb.enable = False
cfg.dataset.image_transforms.enable = True
cfg.log_freq = 20
cfg.num_steps = 2

In [9]:
if cfg.wandb.enable and cfg.wandb.project:
    wandb_logger = WandBLogger(cfg)
else:
    wandb_logger = None
    logging.info(colored("Logs will be saved locally.", "yellow", attrs=["bold"]))

if cfg.seed is not None:
    set_seed(cfg.seed)

device = get_safe_torch_device(cfg.policy.device, log=True)
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

logging.info("Creating dataset")
dataset = make_dataset(cfg)

INFO 2025-12-30 13:01:30 2197283937.py:5 Logs will be saved locally.
INFO 2025-12-30 13:01:30 197283937.py:14 Creating dataset


Resolving data files:   0%|          | 0/200 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

{0: {'episode_index': 0, 'tasks': ['put the black bowl in the bottom drawer of the cabinet and close it'], 'length': 247}, 1: {'episode_index': 1, 'tasks': ['put the black bowl in the bottom drawer of the cabinet and close it'], 'length': 248}, 2: {'episode_index': 2, 'tasks': ['put the black bowl in the bottom drawer of the cabinet and close it'], 'length': 235}, 3: {'episode_index': 3, 'tasks': ['put the black bowl in the bottom drawer of the cabinet and close it'], 'length': 231}, 4: {'episode_index': 4, 'tasks': ['put the black bowl in the bottom drawer of the cabinet and close it'], 'length': 223}, 5: {'episode_index': 5, 'tasks': ['put the yellow and white mug in the microwave and close it'], 'length': 449}, 6: {'episode_index': 6, 'tasks': ['put the yellow and white mug in the microwave and close it'], 'length': 265}, 7: {'episode_index': 7, 'tasks': ['put the yellow and white mug in the microwave and close it'], 'length': 248}, 8: {'episode_index': 8, 'tasks': ['put the yellow 

In [10]:
# Create environment used for evaluating checkpoints during training on simulation data.
# On real-world data, no need to create an environment as evaluations are done outside train.py,
# using the eval.py instead, with gym_dora environment and dora-rs.
eval_env = None
if cfg.eval_freq > 0 and cfg.env is not None:
    logging.info("Creating env")
    eval_env = make_env(cfg.env, n_envs=cfg.eval.batch_size, use_async_envs=cfg.eval.use_async_envs)

In [12]:
cfg.policy.pretrained_path = "/pfss/mlde/workspaces/mlde_wsp_PI_Hauschild/VLA/temp/baseline"
logging.info("Creating policy")
policy = make_policy(
    cfg=cfg.policy,
    ds_meta=dataset.meta,
)


INFO 2025-12-30 13:04:09 3098919642.py:2 Creating policy


load pretrained policy
Loading weights from local directory


In [16]:
cfg.policy.pretrained_path = "../outputs/train/2025-12-29/15-41-19_libero_10%_imi_v8_v3_10-_mh_synthetic_20ep_2_hard_task+5ep_all_normal_task_PRETRAINING_VLM/checkpoints/050000/pretrained_model"
logging.info("Creating policy")
mypolicy = make_policy(
    cfg=cfg.policy,
    ds_meta=dataset.meta,
)


INFO 2025-12-30 13:07:32 3160407189.py:2 Creating policy


load pretrained policy
Loading weights from local directory


In [21]:
import torch

m1 = policy.model.paligemma_with_expert.gemma_expert
m2 = mypolicy.model.paligemma_with_expert.gemma_expert

sd1 = m1.state_dict()
sd2 = m2.state_dict()

assert sd1.keys() == sd2.keys(), "State dict keys mismatch!"

all_equal = True
for k in sd1:
    if not torch.equal(sd1[k], sd2[k]):
        all_equal = False
        print(f"❌ Mismatch at {k}")
        break

print("✅ All parameters identical!" if all_equal else "⚠️ Parameters differ")


✅ All parameters identical!


In [ ]:
policy.model.paligemma_with_expert.gemma_expert


In [19]:
mypolicy.model.paligemma_with_expert.gemma_expert

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): None
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=256, bias=False)
          (v_proj): Linear(in_features=1024, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=1024, out_features=4096, bias=False)
          (up_proj): Linear(in_features=1024, out_features=4096, bias=False)
          (down_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((1024,), eps=1e-06)
      )
    )
    (norm): GemmaRMSNorm((1024,), eps=1e-06)
    (rotary_emb): GemmaRo

In [11]:
logging.info("Creating optimizer and scheduler")
optimizer, lr_scheduler = make_optimizer_and_scheduler(cfg, policy)
grad_scaler = GradScaler(device.type, enabled=cfg.policy.use_amp)

step = 0  # number of policy updates (forward + backward + optim)

if cfg.resume:
    step, optimizer, lr_scheduler = load_training_state(cfg.checkpoint_path, optimizer, lr_scheduler)

num_learnable_params = sum(p.numel() for p in policy.parameters() if p.requires_grad)
num_total_params = sum(p.numel() for p in policy.parameters())

logging.info(colored("Output dir:", "yellow", attrs=["bold"]) + f" {cfg.output_dir}")
if cfg.env is not None:
    logging.info(f"{cfg.env.task=}")
logging.info(f"{cfg.steps=} ({format_big_number(cfg.steps)})")
logging.info(f"{dataset.num_frames=} ({format_big_number(dataset.num_frames)})")
logging.info(f"{dataset.num_episodes=}")
logging.info(f"{num_learnable_params=} ({format_big_number(num_learnable_params)})")
logging.info(f"{num_total_params=} ({format_big_number(num_total_params)})")


INFO 2025-11-21 08:53:04 2797565836.py:1 Creating optimizer and scheduler
INFO 2025-11-21 08:53:04 797565836.py:13 Output dir: outputs/train/2025-11-21/08-52-19_pi0
INFO 2025-11-21 08:53:04 797565836.py:16 cfg.steps=100000 (100K)
INFO 2025-11-21 08:53:04 797565836.py:17 dataset.num_frames=133851 (134K)
INFO 2025-11-21 08:53:04 797565836.py:18 dataset.num_episodes=800
INFO 2025-11-21 08:53:04 797565836.py:19 num_learnable_params=3419097505 (3B)
INFO 2025-11-21 08:53:04 797565836.py:20 num_total_params=3846778283 (4B)


### why not fully finetune, which part is freezed

In [12]:
if hasattr(cfg.policy, "drop_n_last_frames"):
    shuffle = False
    sampler = EpisodeAwareSampler(
        dataset.episode_data_index,
        drop_n_last_frames=cfg.policy.drop_n_last_frames,
        shuffle=True,
    )
else:
    shuffle = True
    sampler = None

dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=cfg.num_workers,
    batch_size=3,
    shuffle=shuffle,
    sampler=sampler,
    pin_memory=device.type != "cpu",
    drop_last=False,
)
dl_iter = cycle(dataloader)


In [ ]:
sample = next(dl_iter)

Parameter 'format_kwargs'={'transform': <function hf_transform_to_torch at 0x152831e75a20>} of the transform datasets.arrow_dataset.Dataset.set_format couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
WARNING 2025-11-21 08:58:29 gerprint.py:258 Parameter 'format_kwargs'={'transform': <function hf_transform_to_torch at 0x152831e75a20>} of the transform datasets.arrow_dataset.Dataset.set_format couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will con

In [14]:
sample['observation.images.image'].shape

torch.Size([3, 3, 256, 256])

In [15]:
def update_policy(
    train_metrics: MetricsTracker,
    policy: PreTrainedPolicy,
    batch: Any,
    optimizer: Optimizer,
    grad_clip_norm: float,
    grad_scaler: GradScaler,
    lr_scheduler=None,
    use_amp: bool = False,
    lock=None,
) -> tuple[MetricsTracker, dict]:
    start_time = time.perf_counter()
    device = get_device_from_parameters(policy)
    policy.train()
    with torch.autocast(device_type=device.type) if use_amp else nullcontext():
        loss, output_dict = policy.forward(batch)
        # TODO(rcadene): policy.unnormalize_outputs(out_dict)
    grad_scaler.scale(loss).backward()

    # Unscale the gradient of the optimizer's assigned params in-place **prior to gradient clipping**.
    grad_scaler.unscale_(optimizer)

    grad_norm = torch.nn.utils.clip_grad_norm_(
        policy.parameters(),
        grad_clip_norm,
        error_if_nonfinite=False,
    )

    # Optimizer's gradients are already unscaled, so scaler.step does not unscale them,
    # although it still skips optimizer.step() if the gradients contain infs or NaNs.
    with lock if lock is not None else nullcontext():
        grad_scaler.step(optimizer)
    # Updates the scale for next iteration.
    grad_scaler.update()

    optimizer.zero_grad()

    # Step through pytorch scheduler at every batch instead of epoch
    if lr_scheduler is not None:
        lr_scheduler.step()

    if has_method(policy, "update"):
        # To possibly update an internal buffer (for instance an Exponential Moving Average like in TDMPC).
        policy.update()

    train_metrics.loss = loss.item()
    train_metrics.grad_norm = grad_norm.item()
    train_metrics.lr = optimizer.param_groups[0]["lr"]
    train_metrics.update_s = time.perf_counter() - start_time
    return train_metrics, output_dict


In [17]:

policy.train()

train_metrics = {
    "loss": AverageMeter("loss", ":.3f"),
    "grad_norm": AverageMeter("grdn", ":.3f"),
    "lr": AverageMeter("lr", ":0.1e"),
    "update_s": AverageMeter("updt_s", ":.3f"),
    "dataloading_s": AverageMeter("data_s", ":.3f"),
}

train_tracker = MetricsTracker(
    cfg.batch_size, dataset.num_frames, dataset.num_episodes, train_metrics, initial_step=step
)

logging.info("Start offline training on a fixed dataset")
for _ in range(step, cfg.steps):
    start_time = time.perf_counter()
    batch = next(dl_iter)
    train_tracker.dataloading_s = time.perf_counter() - start_time

    for key in batch:
        if isinstance(batch[key], torch.Tensor):
            batch[key] = batch[key].to(device, non_blocking=True)

    train_tracker, output_dict = update_policy(
        train_tracker,
        policy,
        batch,
        optimizer,
        cfg.optimizer.grad_clip_norm,
        grad_scaler=grad_scaler,
        lr_scheduler=lr_scheduler,
        use_amp=cfg.policy.use_amp,
    )

    # Note: eval and checkpoint happens *after* the `step`th training update has completed, so we
    # increment `step` here.
    step += 1
    train_tracker.step()
    is_log_step = cfg.log_freq > 0 and step % cfg.log_freq == 0
    is_saving_step = step % cfg.save_freq == 0 or step == cfg.steps
    is_eval_step = cfg.eval_freq > 0 and step % cfg.eval_freq == 0

    if is_log_step:
        ###### compute l1 loss
        print("ductop1")
        pred = policy.select_action_chunk(batch)
        print(pred[:,-1].min(), pred[:,-1].max())
        pred[:,:,-1] = torch.where(pred[:,:,-1] > 0, torch.tensor(1.0, device=device), torch.tensor(-1.0, device=device))
        gt = batch[ACTION]
        print(gt[:,-1].min(), gt[:,-1].max())
        assert pred.shape == gt.shape, f"Pred shape {pred.shape} does not match GT shape {gt.shape}"
        l1_infer_loss = torch.nn.functional.l1_loss(pred, gt)
        ######

        logging.info(train_tracker)
        if wandb_logger:
            wandb_log_dict = train_tracker.to_dict()
            if output_dict:
                wandb_log_dict.update(output_dict)
            wandb_logger.log_dict(wandb_log_dict, step)
        train_tracker.reset_averages()

    if cfg.save_checkpoint and is_saving_step:
        logging.info(f"Checkpoint policy after step {step}")
        checkpoint_dir = get_step_checkpoint_dir(cfg.output_dir, cfg.steps, step)
        save_checkpoint(checkpoint_dir, step, cfg, policy, optimizer, lr_scheduler)
        update_last_checkpoint(checkpoint_dir)
        if wandb_logger:
            wandb_logger.log_policy(checkpoint_dir)
        print(wandb_logger)

    if cfg.env and is_eval_step:
        step_id = get_step_identifier(step, cfg.steps)
        logging.info(f"Eval policy at step {step}")
        with (
            torch.no_grad(),
            torch.autocast(device_type=device.type) if cfg.policy.use_amp else nullcontext(),
        ):
            eval_info = eval_policy(
                eval_env,
                policy,
                cfg.eval.n_episodes,
                videos_dir=cfg.output_dir / "eval" / f"videos_step_{step_id}",
                max_episodes_rendered=4,
                start_seed=cfg.seed,
            )

        eval_metrics = {
            "avg_sum_reward": AverageMeter("∑rwrd", ":.3f"),
            "pc_success": AverageMeter("success", ":.1f"),
            "eval_s": AverageMeter("eval_s", ":.3f"),
        }
        eval_tracker = MetricsTracker(
            cfg.batch_size, dataset.num_frames, dataset.num_episodes, eval_metrics, initial_step=step
        )
        eval_tracker.eval_s = eval_info["aggregated"].pop("eval_s")
        eval_tracker.avg_sum_reward = eval_info["aggregated"].pop("avg_sum_reward")
        eval_tracker.pc_success = eval_info["aggregated"].pop("pc_success")
        logging.info(eval_tracker)
        if wandb_logger:
            wandb_log_dict = {**eval_tracker.to_dict(), **eval_info}
            wandb_logger.log_dict(wandb_log_dict, step, mode="eval")
            wandb_logger.log_video(eval_info["video_paths"][0], step, mode="eval")

if eval_env:
    eval_env.close()
logging.info("End of training")


INFO 2025-11-21 08:55:38 085197248.py:15 Start offline training on a fixed dataset


ductop1


INFO 2025-11-21 08:55:48 085197248.py:56 step:40 smpl:40 ep:0 epch:0.00 loss:6.846 grdn:247.396 lr:3.1e-06 updt_s:0.482 data_s:0.000


tensor(-0.1520, device='cuda:0') tensor(0.5869, device='cuda:0')
tensor(-0.3402, device='cuda:0') tensor(1., device='cuda:0')
ductop1


INFO 2025-11-21 08:55:58 085197248.py:56 step:60 smpl:60 ep:0 epch:0.00 loss:3.874 grdn:110.192 lr:5.1e-06 updt_s:0.483 data_s:0.000


...

INFO 2025-11-21 08:58:18 085197248.py:56 step:340 smpl:340 ep:2 epch:0.00 loss:1.906 grdn:59.301 lr:3.3e-05 updt_s:0.478 data_s:0.000


tensor(-0.4432, device='cuda:0') tensor(0.5735, device='cuda:0')
tensor(-0.6402, device='cuda:0') tensor(1., device='cuda:0')
ductop1


INFO 2025-11-21 08:58:28 085197248.py:56 step:360 smpl:360 ep:2 epch:0.00 loss:2.122 grdn:40.127 lr:3.5e-05 updt_s:0.487 data_s:0.000


tensor(-0.7838, device='cuda:0') tensor(0.8872, device='cuda:0')
tensor(-0.1854, device='cuda:0') tensor(1., device='cuda:0')


KeyboardInterrupt: 